In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('./processed/train2.csv', index_col=0)
test = pd.read_csv('./processed/test2.csv', index_col=0)

In [ ]:
train.head(3)

In [ ]:
test.head(3)

## 3.1.連続値の離散化

　qcut関数を用いて、連続値の離散化を行う。

In [ ]:
df1 = pd.qcut(train['Fare'], 4)
pd.value_counts(df1)

In [ ]:
datasets = [train, test]

for dataset in datasets:
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
sns.barplot(x = 'FareBin', y = 'Survived', data=train)

In [ ]:
fare_kde = sns.FacetGrid(train, hue='Survived', aspect=4)
fare_kde.map(sns.kdeplot, 'Fare', shade=True )
fare_kde.set(xlim=(0 , 50))
fare_kde.add_legend()

　　scikit-learnの<a href="https://github.com/scikit-learn/scikit-learn/tree/master/sklearn/preprocessing">preprocessing</a>フォルダにある<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/preprocessing/label.py#L40">LabelEncoder</a>クラスを用いることで、離散的な値を整数値で表現できる。

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
datasets = [train, test]
label = LabelEncoder()

for dataset in datasets:    
    dataset['FareLabel'] = label.fit_transform(dataset['FareBin'])

　同様にAgeも離散化する。

In [ ]:
datasets = [train, test]

for dataset in datasets:
    dataset['AgeBin'] = pd.qcut(dataset['Age'], 4)

In [ ]:
sns.barplot(x = 'AgeBin', y = 'Survived', data=train)

　各階層の生存率は0.3から0.4前後に集中している。これは離散化前の分布を反映しているだろうか。

In [ ]:
age_kde = sns.FacetGrid(train, hue='Survived', aspect=4 )
age_kde.map(sns.kdeplot, 'Age', shade=True )
age_kde.set(xlim=(0 , train['Age'].max()))
age_kde.add_legend()

In [ ]:
datasets = [train, test]

for dataset in datasets:
    dataset.drop(['AgeBin'], axis=1, inplace=True)

　生存率を反映するよう離散化する。

　※参考https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python

In [ ]:
datasets = [train, test]

for dataset in datasets:
    dataset.loc[ dataset['Age'] <= 16, 'AgeLabel'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'AgeLabel'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'AgeLabel'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'AgeLabel'] = 3
    dataset.loc[ dataset['Age'] > 64, 'AgeLabel'] = 4
    dataset['AgeLabel'] = dataset['AgeLabel'].astype(int)

In [ ]:
sns.barplot(x = 'AgeLabel', y = 'Survived', data=train)

In [ ]:
train.info()

　データの前処理は完了した。機械学習する前に、不要な属性を削除しておく。

In [ ]:
columns = ['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'FareBin']

train_1 = train.drop(columns, axis=1)
test_1 = test.drop(columns, axis=1)

In [ ]:
train_1.head(3)

In [ ]:
test_1.head(3)

　加工したデータをCSVファイルで保存しておく。

In [ ]:
train_1.to_csv('./processed/train3.csv')
test_1.to_csv('./processed/test3.csv')

## 3.2.機械学習

　scikit-learnには<a href="https://github.com/scikit-learn/scikit-learn/tree/master/sklearn/model_selection">model_selection</a>というフォルダが用意されている。model_selectionフォルダの<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_split.py#L1916">train_test_split</a>関数を用いることで、データをランダムに分割することができる。今回は与えられた訓練データtrainを分割し、新たに訓練データ(train_x, train_y)とテストデータ(test_x, test_y)を用意する。訓練データ(train_x, train_y)を用いて機械学習モデルの訓練を行い、テストデータ(test_x, test_y)を用いて得られたモデルの性能を評価する。なおtrain_xとtest_xは属性のDataFrameであり、test_yとtest_yはクラスのSeriesである。
 
 ![](https://www.developer.com/imagesvr_ce/6793/ML4.png)
 
 [https://www.developer.com/mgmt/real-world-machine-learning-model-evaluation-and-optimization.html](https://www.developer.com/mgmt/real-world-machine-learning-model-evaluation-and-optimization.html)

In [ ]:
from sklearn import model_selection

In [ ]:
train_x, test_x, train_y, test_y = \
model_selection.train_test_split(train_1.drop(['PassengerId', 'Survived'], axis=1), train_1['Survived'], random_state = 0)

In [ ]:
train_x.head(3)

In [ ]:
train_y.head(3)

### 3.2.1.決定木

　scikit-learnには<a href="https://github.com/scikit-learn/scikit-learn/tree/master/sklearn/tree">tree</a>というフォルダが用意されている。treeフォルダの<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/tree/tree.py#L518">DecisionTreeClassifier</a>クラスを用いることで、機械学習アルゴリズムのひとつである決定木を用いることができる。

In [ ]:
from sklearn import tree

　アルゴリズムを利用するために、まずはDecisionTreeClassifierクラスのインスタンスを生成する。

In [ ]:
dtree = tree.DecisionTreeClassifier()

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/tree/tree.py#L752">fit</a>メソッドを用いることで、訓練データを与えて学習を行い、予測モデルを生成できる。

In [ ]:
model = dtree.fit(train_x, train_y)

　生成したモデルを評価する前に、可視化してみよう。Graphvizというツールを用いることで、DOT言語からグラフを生成することができる。そしてPyDotPlusというライブラリを用いることで、PythonでDOT言語を扱うことができる。

In [ ]:
!brew install graphviz

In [ ]:
!pip install pydotplus
#!pip3 install pydotplus

　scikit-learnの<a href="https://github.com/scikit-learn/scikit-learn/blob/a24c8b46/sklearn/tree/export.py#L74">export_graphviz</a>メソッドを用いることで、生成した決定木をDOT言語のフォーマットで出力することができる。

In [ ]:
import pydotplus
from sklearn.externals.six import StringIO

dot_data = StringIO()

tree.export_graphviz(model,
                     filled=True, 
                     feature_names=train_1.drop(['PassengerId', 'Survived'], axis=1).columns[:],
                     class_names=True,
                     out_file=dot_data
                    )

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

graph.write_png("DecisionTree.png")

In [ ]:
from IPython.display import Image

Image(graph.create_png())

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/tree/tree.py#L498">feature_importances</a>メソッドを用いることで、各属性の重要度を定量的に評価できる。

In [ ]:
fti = model.feature_importances_
name = train_x.columns.values

pd.Series(fti, index=name).sort_values(ascending=False)

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/tree/tree.py#L391">predict</a>メソッドを用いることで、クラスの予測を行うことができる。

In [ ]:
pred = model.predict(test_x)

　機械学習におけるモデルの評価基準のひとつである精度(Accuracy)を出してみよう。scikit-learnには<a href="https://github.com/scikit-learn/scikit-learn/tree/master/sklearn/metrics">metrics</a>というフォルダが用意されている。metricsフォルダの<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/metrics/classification.py#L115">accuracy_score</a>関数を用いることで、モデルの精度を求めることができる。

In [ ]:
from sklearn import metrics

In [ ]:
score = metrics.accuracy_score(test_y, pred)
print(score)

### 3.2.2.ランダムフォレスト

　scikit-learnには<a href="https://github.com/scikit-learn/scikit-learn/tree/master/sklearn/ensemble">ensemble</a>というフォルダが用意されている。ensembleフォルダの<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/ensemble/forest.py#L744">RandomForestClassifier</a>クラスを用いることで、機械学習アルゴリズムのひとつである決定木を用いることができる。

In [ ]:
from sklearn import ensemble

　アルゴリズムを利用するために、まずはRandomForestClassifierクラスのインスタンスを生成する。

In [ ]:
forest = ensemble.RandomForestClassifier()

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/ensemble/forest.py#L220">fit</a>メソッドを用いることで、訓練データを与えて学習を行い、予測モデルを生成できる。

In [ ]:
model = forest.fit(train_x, train_y)

　scikit-learnのランダムフォレストは<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/ensemble/forest.py#L974">デフォルトで10個</a>の決定木を生成する。今回はそのうち3個の決定木を可視化してみよう。

In [ ]:
graph = []

for i in range(3):
    dot_data = StringIO()
    tree.export_graphviz(model.estimators_[i],
                     filled=True, 
                     feature_names=train_1.drop(['PassengerId', 'Survived'], axis=1).columns[:],
                     class_names=True,
                     out_file=dot_data
                    )
    graph.append(pydotplus.graph_from_dot_data(dot_data.getvalue()))

In [ ]:
Image(graph[0].create_png())

In [ ]:
Image(graph[1].create_png())

In [ ]:
Image(graph[2].create_png())

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/ensemble/forest.py#L359">feature_importances</a>メソッドを用いることで、各属性の重要度を定量的に評価できる。

In [ ]:
fti = model.feature_importances_
name = train_x.columns.values

pd.Series(fti, index=name).sort_values(ascending=False)

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/ensemble/forest.py#L517">predict</a>メソッドを用いることで、クラスの予測を行うことができる。

In [ ]:
pred = forest.predict(test_x)

In [ ]:
score = metrics.accuracy_score(test_y, pred)
print(score)

### 3.2.3.サポートベクターマシン

　scikit-learnには<a href="https://github.com/scikit-learn/scikit-learn/tree/master/sklearn/svm">svm</a>というフォルダが用意されている。svmフォルダの<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/svm/classes.py#L427">SVC</a>クラスを用いることで、機械学習アルゴリズムのひとつであるサポートベクターマシンを用いることができる。

In [ ]:
from sklearn import svm

　アルゴリズムを利用するために、まずはSVCクラスのインスタンスを生成する。

In [ ]:
svc = svm.SVC()

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/svm/base.py#L108">fit</a>メソッドを用いることで、訓練データを与えて学習を行い、予測モデルを生成できる。

In [ ]:
model = svc.fit(train_x, train_y)

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/svm/base.py#L307">predict</a>メソッドを用いることで、クラスの予測を行うことができる。

In [ ]:
pred = model.predict(test_x)

　精度(Accuracy)を出してみよう。

In [ ]:
score = metrics.accuracy_score(test_y, pred)
print(score)

### 3.2.4.k近傍法

　scikit-learnには<a href="https://github.com/scikit-learn/scikit-learn/tree/a24c8b464d094d2c468a16ea9f8bf8d42d949f84/sklearn/neighbors">neighbors</a>というフォルダが用意されている。neighborsフォルダの<a href="https://github.com/scikit-learn/scikit-learn/blob/a24c8b46/sklearn/neighbors/classification.py#L23">KNeighborsClassifier</a>クラスを用いることで、機械学習アルゴリズムのひとつであるk近傍法を用いることができる。

In [ ]:
#プログラムをインポートする。

　アルゴリズムを利用するために、まずはKNeighborsClassifierクラスのインスタンスを生成する。

In [ ]:
#KNeighborsClassifierクラスのインスタンスを生成する。

　<a href="https://github.com/scikit-learn/scikit-learn/blob/a24c8b464d094d2c468a16ea9f8bf8d42d949f84/sklearn/neighbors/base.py#L751">fit</a>メソッドを用いることで、訓練データを与えて学習を行い、予測モデルを生成できる。

In [ ]:
#fitメソッドを用いることで、訓練データを与えて学習を行う。

　<a href="https://github.com/scikit-learn/scikit-learn/blob/a24c8b464d094d2c468a16ea9f8bf8d42d949f84/sklearn/neighbors/classification.py#L129">predict</a>メソッドを用いることで、クラスの予測を行うことができる

In [ ]:
#predictメソッドを用いることで、クラスの予測を行う。

　精度(Accuracy)を出してみよう。

In [ ]:
#精度(Accuracy)を出す。

### 3.2.5.パーセプトロン

　scikit-learnには<a href="https://github.com/scikit-learn/scikit-learn/tree/master/sklearn/linear_model">linear_model</a>というフォルダが用意されている。linear_modelフォルダの<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/linear_model/perceptron.py#L7">Perceptron</a>クラスを用いることで、機械学習アルゴリズムのひとつであるパーセプトロンを用いることができる。

In [ ]:
#プログラムをインポートする。

　アルゴリズムを利用するために、まずはPerceptronクラスのインスタンスを生成する。

In [ ]:
#Perceptronクラスのインスタンスを生成する。

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/linear_model/stochastic_gradient.py#L559">fit</a>メソッドを用いることで、訓練データを与えて学習を行い、予測モデルを生成できる。

In [ ]:
#fitメソッドを用いることで、訓練データを与えて学習を行う。

　<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/linear_model/stochastic_gradient.py#L1090">predict</a>メソッドを用いることで、クラスの予測を行うことができる。

In [ ]:
#predictメソッドを用いることで、クラスの予測を行う。

　精度(Accuracy)を出してみよう。

In [ ]:
#精度(Accuracy)を出す。